In [2]:
import os

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\ramak\\OneDrive\\Desktop\\P2\\Birds_Classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelPreparationConfig:
    root_dir: Path
    saved_model_dir: Path
    input_shape: int
    include_top: bool
    weights: str
    classes: int
    learning_rate: float

In [7]:
# dir_train = "C:\\Users\\ramak\\OneDrive\\Desktop\\P2\\Birds_Classification\\artifacts\\data_ingestion\\train/"
dir_train = os.path.join("artifacts","data_ingestion","train/")
classes = os.listdir(dir_train)
print(classes)
n_classes = len(classes)
print("Total Classes: ",n_classes)

['ABBOTTS BABBLER', 'ABBOTTS BOOBY', 'ABYSSINIAN GROUND HORNBILL', 'AFRICAN CROWNED CRANE', 'AFRICAN EMERALD CUCKOO', 'AFRICAN FIREFINCH', 'AFRICAN OYSTER CATCHER', 'AFRICAN PIED HORNBILL', 'AFRICAN PYGMY GOOSE', 'ALBATROSS', 'ALBERTS TOWHEE', 'ALEXANDRINE PARAKEET', 'ALPINE CHOUGH', 'ALTAMIRA YELLOWTHROAT', 'AMERICAN AVOCET', 'AMERICAN BITTERN', 'AMERICAN COOT', 'AMERICAN DIPPER', 'AMERICAN FLAMINGO', 'AMERICAN GOLDFINCH', 'AMERICAN KESTREL', 'AMERICAN PIPIT', 'AMERICAN REDSTART', 'AMERICAN ROBIN', 'AMERICAN WIGEON', 'AMETHYST WOODSTAR', 'ANDEAN GOOSE', 'ANDEAN LAPWING', 'ANDEAN SISKIN', 'ANHINGA', 'ANIANIAU', 'ANNAS HUMMINGBIRD', 'ANTBIRD', 'ANTILLEAN EUPHONIA', 'APAPANE', 'APOSTLEBIRD', 'ARARIPE MANAKIN', 'ASHY STORM PETREL', 'ASHY THRUSHBIRD', 'ASIAN CRESTED IBIS', 'ASIAN DOLLARD BIRD', 'ASIAN GREEN BEE EATER', 'ASIAN OPENBILL STORK', 'AUCKLAND SHAQ', 'AUSTRAL CANASTERO', 'AUSTRALASIAN FIGBIRD', 'AVADAVAT', 'AZARAS SPINETAIL', 'AZURE BREASTED PITTA', 'AZURE JAY', 'AZURE TANAGER', '

In [5]:
from src.BirdImageClassification.constants import *
from src.BirdImageClassification.utils.common import read_yaml,create_directory
from src.BirdImageClassification import logger

True 
 True


In [6]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directory([self.config.main_dir])


    def base_model_config(self) -> ModelPreparationConfig:
        config = self.config.model_preparation
        create_directory([config.root_dir])

        model_preparation_config = ModelPreparationConfig(
            root_dir = Path(config.root_dir),
            saved_model_dir = Path(config.model_saved_dir),
            input_shape = self.params.INPUT_SHAPE,
            include_top = self.params.INCLUDE_TOP,
            weights = self.params.WEIGHTS,
            classes = self.params.CLASSES,
            learning_rate = self.params.LEARNING_RATE
        )

        return model_preparation_config



In [7]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3, VGG16, DenseNet201
from tensorflow.keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input,Flatten, Dropout,GlobalMaxPooling2D,Lambda,GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

[2023-12-22 12:16:51,941: INFO: utils: NumExpr defaulting to 8 threads.]


In [8]:
class ModelPreparation:
    def __init__(self, config: ModelPreparationConfig):
        self.config = config
        print(self.config.input_shape)
        self.base_model = DenseNet201(weights=self.config.weights, include_top=self.config.include_top,
                                      input_shape=(self.config.input_shape,self.config.input_shape,3))

    def prepare_base_model(self):
        model = Sequential()
        model.add(self.base_model)

        for layer in self.base_model.layers:
            layer.trainable = False

        model.add(GlobalAveragePooling2D())
        model.add(Dense(512, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(256, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        model.add(Dense(self.config.classes, activation='softmax'))

        # lr = 0.005 

        optimizers = tf.keras.optimizers.Adam(learning_rate=self.config.learning_rate)

        model.compile(loss="categorical_crossentropy",
                      optimizer=optimizers,
                      metrics=["accuracy"])

        model.summary()

        logger.info("Model saving................................")
        self.save_model(path=self.config.saved_model_dir, model=model)

    def save_model(self, path: Path, model=tf.keras.Model):
        model.save(path)
        logger.info("Model Saved.............................")

In [9]:
try:
    config = ConfigurationManager()
    model_preparation_config = config.base_model_config()
    model_preparation = ModelPreparation(config=model_preparation_config)
    # model_preparation.get_prepared_model()
    model_preparation.prepare_base_model()
except Exception as e:
    raise e

[2023-12-22 12:16:53,709: INFO: common: yaml file: C:\Users\ramak\OneDrive\Desktop\P2\Birds_Classification\config\config.yaml loaded]
[2023-12-22 12:16:53,717: INFO: common: yaml file: C:\Users\ramak\OneDrive\Desktop\P2\Birds_Classification\params.yaml loaded]
[2023-12-22 12:16:53,725: INFO: common: Created directory at: artifacts]
[2023-12-22 12:16:53,731: INFO: common: Created directory at: artifacts/model_preparation]


150
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization_94 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 525)            

C:\Users\ramak\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[2023-12-22 12:17:00,162: INFO: 499976278: Model Saved.............................]
